# Spending Policies Comparison

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/engineerinvestor/monteplan/blob/main/notebooks/02_spending_policies.ipynb)

This notebook compares all 5 spending policies in monteplan side-by-side using the same plan and market assumptions.

**Key takeaway:** There is no free lunch -- higher success rates come at the cost of spending volatility.

In [ ]:
# Uncomment and run in Google Colab:
# !pip install -q "monteplan @ git+https://github.com/engineerinvestor/monteplan.git"

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from monteplan.config.defaults import default_plan, default_market, default_sim_config
from monteplan.config.schema import (
    PolicyBundle, SpendingPolicyConfig, SimulationConfig,
    GuardrailsConfig, VPWConfig, FloorCeilingConfig,
)
from monteplan.core.engine import simulate

In [ ]:
# Standard plan and market for fair comparison
plan = default_plan()
market = default_market()
sim_config = SimulationConfig(n_paths=1000, seed=42)  # fast preset for quick comparison

print(f"Plan: age {plan.current_age} -> retire {plan.retirement_age} -> {plan.end_age}")
print(f"Monthly spending: ${plan.monthly_spending:,.0f}")
print(f"Simulation: {sim_config.n_paths} paths, seed={sim_config.seed}")

## 1. Constant Real Spending (4% Rule)

Withdraw a fixed real dollar amount each month, adjusted only for inflation. The classic approach.

In [ ]:
policies_constant = PolicyBundle(
    spending=SpendingPolicyConfig(policy_type="constant_real"),
)
result_constant = simulate(plan, market, policies_constant, sim_config)
print(f"Constant Real -- Success: {result_constant.success_probability:.1%}")

## 2. Percent of Portfolio

Withdraw a fixed percentage of the current portfolio value each month. Spending fluctuates with the market, but the portfolio never fully depletes.

In [ ]:
policies_pct = PolicyBundle(
    spending=SpendingPolicyConfig(policy_type="percent_of_portfolio", withdrawal_rate=0.04),
)
result_pct = simulate(plan, market, policies_pct, sim_config)
print(f"Percent of Portfolio -- Success: {result_pct.success_probability:.1%}")

## 3. Guyton-Klinger Guardrails

Starts with an initial withdrawal rate, then adjusts up/down based on how the actual withdrawal rate compares to the initial rate. Includes "prosperity" (raise) and "capital preservation" (cut) rules.

In [ ]:
policies_guardrails = PolicyBundle(
    spending=SpendingPolicyConfig(
        policy_type="guardrails",
        guardrails=GuardrailsConfig(
            initial_withdrawal_rate=0.05,
            upper_threshold=0.20,
            lower_threshold=0.20,
            raise_pct=0.10,
            cut_pct=0.10,
        ),
    ),
)
result_guardrails = simulate(plan, market, policies_guardrails, sim_config)
print(f"Guardrails -- Success: {result_guardrails.success_probability:.1%}")

## 4. Variable Percentage Withdrawal (VPW)

Withdraw 1/remaining_years of the portfolio, bounded by min/max rates. Naturally increases the withdrawal rate as you age.

In [ ]:
policies_vpw = PolicyBundle(
    spending=SpendingPolicyConfig(
        policy_type="vpw",
        vpw=VPWConfig(min_rate=0.03, max_rate=0.15),
    ),
)
result_vpw = simulate(plan, market, policies_vpw, sim_config)
print(f"VPW -- Success: {result_vpw.success_probability:.1%}")

## 5. Floor and Ceiling

Withdraw a percentage of the portfolio, clamped between an inflation-adjusted floor and ceiling.

In [ ]:
policies_fc = PolicyBundle(
    spending=SpendingPolicyConfig(
        policy_type="floor_ceiling",
        floor_ceiling=FloorCeilingConfig(
            withdrawal_rate=0.04,
            floor=3_000,
            ceiling=10_000,
        ),
    ),
)
result_fc = simulate(plan, market, policies_fc, sim_config)
print(f"Floor & Ceiling -- Success: {result_fc.success_probability:.1%}")

## Comparison Table

In [ ]:
results = [
    ("Constant Real", result_constant),
    ("Percent of Portfolio", result_pct),
    ("Guardrails", result_guardrails),
    ("VPW", result_vpw),
    ("Floor & Ceiling", result_fc),
]

rows = []
for name, res in results:
    rows.append({
        "Policy": name,
        "Success %": f"{res.success_probability:.1%}",
        "Median Terminal": f"${res.terminal_wealth_percentiles['p50']:,.0f}",
        "P5 Terminal": f"${res.terminal_wealth_percentiles['p5']:,.0f}",
        "P95 Terminal": f"${res.terminal_wealth_percentiles['p95']:,.0f}",
    })

df = pd.DataFrame(rows)
df

## Overlaid Median Wealth Paths

In [ ]:
colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd"]

fig, ax = plt.subplots(figsize=(10, 5))
for (name, res), color in zip(results, colors):
    ts = res.wealth_time_series
    ages = np.linspace(plan.current_age, plan.end_age, len(ts["p50"]))
    ax.plot(ages, ts["p50"], label=name, color=color, linewidth=2)

ax.axvline(plan.retirement_age, color="gray", linestyle="--", alpha=0.5)
ax.set_xlabel("Age")
ax.set_ylabel("Median Portfolio Value ($)")
ax.set_title("Median Wealth by Spending Policy")
ax.legend()
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"${x/1e6:.1f}M" if x >= 1e6 else f"${x/1e3:.0f}K"))
plt.tight_layout()
plt.show()

## Overlaid Median Spending Paths

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
for (name, res), color in zip(results, colors):
    ts = res.spending_time_series
    ages = np.linspace(plan.current_age, plan.end_age, len(ts["p50"]))
    # Only show retirement period
    ret_idx = int((plan.retirement_age - plan.current_age) / (plan.end_age - plan.current_age) * len(ages))
    ax.plot(ages[ret_idx:], ts["p50"][ret_idx:], label=name, color=color, linewidth=2)

ax.set_xlabel("Age")
ax.set_ylabel("Median Monthly Spending ($)")
ax.set_title("Median Spending by Policy (Retirement Only)")
ax.legend()
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"${x:,.0f}"))
plt.tight_layout()
plt.show()

## Key Takeaways

1. **Constant real spending** is the simplest but most rigid -- it ignores portfolio performance
2. **Percent of portfolio** never depletes but spending can drop dramatically in downturns
3. **Guardrails** offer a good balance -- adaptive spending with bounded changes
4. **VPW** is actuarially sound and naturally spends down the portfolio
5. **Floor and ceiling** guarantees a minimum spending level but can still deplete

There is no universally best policy. The right choice depends on your priorities:
- **Spending stability** -> Constant real or floor/ceiling
- **Portfolio survival** -> Guardrails or VPW
- **Simplicity** -> Constant real or percent of portfolio